# MNIST 模型強化

## 步驟1：載入 MNIST 手寫阿拉伯數字資料

In [6]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

# 載入 MNIST 手寫阿拉伯數字資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

## 步驟2：改用 CNN 模型

In [7]:
# 建立模型
from tensorflow.keras import layers
import numpy as np

# 增加一維在最後面
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# CNN 模型
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=x_train.shape[1:]),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 步驟3：資料增補(Data Augmentation)

In [15]:
# 參數設定
batch_size = 1000
epochs = 5

# 資料增補定義
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,         # 特徵縮放
        rotation_range=10,      # 旋轉 10 度
        zoom_range=0.1,         # 拉遠/拉近 10%
        width_shift_range=0.1,  # 寬度偏移  10%
        height_shift_range=0.1) # 高度偏移  10%

# 增補資料，進行模型訓練
datagen.fit(x_train)
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
          validation_data=datagen.flow(x_test, y_test, batch_size=batch_size), verbose=2)

C:\Users\mikec\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
60/60 - 19s - 309ms/step - accuracy: 0.9059 - loss: 0.3084 - val_accuracy: 0.9411 - val_loss: 0.2032
Epoch 2/5
60/60 - 18s - 307ms/step - accuracy: 0.9186 - loss: 0.2692 - val_accuracy: 0.9520 - val_loss: 0.1692
Epoch 3/5
60/60 - 18s - 306ms/step - accuracy: 0.9275 - loss: 0.2412 - val_accuracy: 0.9566 - val_loss: 0.1568
Epoch 4/5
60/60 - 19s - 309ms/step - accuracy: 0.9331 - loss: 0.2187 - val_accuracy: 0.9583 - val_loss: 0.1402
Epoch 5/5
60/60 - 19s - 310ms/step - accuracy: 0.9399 - loss: 0.1997 - val_accuracy: 0.9620 - val_loss: 0.1255


In [16]:
# 評分(Score Model)
score=model.evaluate(x_test, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

loss: 9.6411
compile_metrics: 0.9767


## 步驟4：測試自行繪製的數字

In [17]:
# 使用小畫家，繪製 0~9，實際測試看看
from skimage import io
from skimage.transform import resize
import numpy as np

# 讀取影像並轉為單色
uploaded_file = '../myDigits/9.png'
image1 = io.imread(uploaded_file, as_gray=True)

# 縮為 (28, 28) 大小的影像
image_resized = resize(image1, (28, 28))    
X1 = image_resized.reshape(1,28, 28, 1) #/ 255

# 反轉顏色，顏色0為白色，與 RGB 色碼不同，它的 0 為黑色
X1 = np.abs(1-X1)

# 預測
predictions = np.argmax(model.predict(X1, verbose=False), axis=-1)
print(predictions[0])

9
